# import packages

In [15]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [16]:
import pandas as pd
import scipy.stats as stats

# load dataset

Read the CSV data into a DataFrame, define -1 values as None

In [17]:
path = "../data/dft-road-casualty-statistics-casualty-provisional-mid-year-unvalidated-2022.csv"
df = pd.read_csv(path, na_values=-1)

# data exploration

In [ ]:
# Shape of the dataset
shape = df.shape

# Data types of the columns
data_types = df.dtypes

# Check for missing values
missing_values = df.isnull().sum()

# Descriptive statistics of the numerical columns
descriptive_stats = df.describe()

# Count of unique values for each column
unique_counts = df.nunique()

shape, data_types, missing_values, descriptive_stats, unique_counts

Check for outliers or inconsistencies For numerical columns and boxplot

In [ ]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Set up subplots
fig, axes = plt.subplots(nrows=1, ncols=len(numerical_columns), figsize=(15, 5))

# Plot boxplots for each numerical column
for i, column in enumerate(numerical_columns):
    df.boxplot(column=column, ax=axes[i])
    axes[i].set_xlabel(column, rotation=90, labelpad=10)  # Rotate x-axis labels by 90 degrees and place them above
    axes[i].tick_params(axis='x', bottom=False)  # Hide x-axis ticks

plt.tight_layout()
plt.show()


Check for outliers or inconsistencies For categorical columns

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    # Check unique values
    print(f"\nUnique values in {column}:")
    print(df[column].value_counts())

Calculate the range of diversity for each numerical column

In [ ]:
diversity_ranges = {}
for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:  # Check if the column is numerical
        column_range = df[column].max() - df[column].min()
        diversity_ranges[column] = column_range + 1

# Print the range of diversity for each numerical column
for column, diversity_range in diversity_ranges.items():
    print(f"Range of diversity for '{column}': {diversity_range}")

# Descriptive Statistics

Summary statistics for numerical variables

In [ ]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
numerical_summary = df[numerical_columns].describe()
print("Summary statistics for numerical variables:")
print(numerical_summary)

Frequency tables for categorical variables

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f"\nFrequency table for {column}:")
    print(df[column].value_counts())

# Data preprocessing

In [ ]:
df_numeric = df.select_dtypes(exclude=['object'])
# Calculate correlation matrix
correlation_matrix = df_numeric.corr()

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Plot correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap=cmap, cbar_kws={"shrink": .5}, fmt=".2f")
plt.title('Correlation Matrix of Numerical Variables')
plt.show()

# Data Cleaning

In [18]:
df.drop(columns=['accident_year', 'accident_reference', 'status'], inplace=True)
df.drop_duplicates(inplace=True)

Handle missing data

In [19]:
from sklearn.impute import SimpleImputer

# Impute missing values with mode
imputer_mode = SimpleImputer(strategy='most_frequent')

# Fit the imputer and transform the data
df['sex_of_casualty'] = imputer_mode.fit_transform(df[['sex_of_casualty']])
df['casualty_type'] = imputer_mode.fit_transform(df[['casualty_type']])
df['car_passenger'] = imputer_mode.fit_transform(df[['car_passenger']])
df['bus_or_coach_passenger'] = imputer_mode.fit_transform(
    df[['bus_or_coach_passenger']])
df['pedestrian_road_maintenance_worker'] = imputer_mode.fit_transform(
    df[['pedestrian_road_maintenance_worker']])

# Impute missing values with median
imputer_median = SimpleImputer(strategy='median')

df['age_of_casualty'] = imputer_median.fit_transform(df[['age_of_casualty']])
df['age_band_of_casualty'] = imputer_median.fit_transform(
    df[['age_band_of_casualty']])
df.dropna(inplace=True)


remove outliers

In [20]:
df = df[(df['vehicle_reference'] != 227) & (df['vehicle_reference'] != 61)]
df = df[(df['casualty_reference'] != 22) & (df['casualty_reference'] != 148)]


# Data Type Conversion

In [21]:
columns_to_convert = ['sex_of_casualty', 'age_of_casualty',
                      'age_band_of_casualty',
                      'car_passenger', 'bus_or_coach_passenger',
                      'pedestrian_road_maintenance_worker', 'casualty_type',
                      'casualty_home_area_type', 'casualty_imd_decile']

# Convert columns to int type
df[columns_to_convert] = df[columns_to_convert].astype(int)

# Exploratory Data Analysis (EDA)

Plot the distribution of accident severities

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

current_palette = sns.color_palette() 

plt.figure(figsize=(10, 6))
sns.countplot(x='casualty_severity', data=df, palette=current_palette)

plt.title('Distribution of Accident Severities', fontsize=16)
plt.xlabel('Casualty Severity', fontsize=14)
plt.ylabel('Count', fontsize=14)

plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


Compare the distribution of casualty severity across different demographic groups

1. Gender vs. Casualty Severity

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='casualty_severity', hue='sex_of_casualty', data=df, palette=current_palette)
plt.title('Casualty Severity by Gender')
plt.xlabel('Casualty Severity')
plt.ylabel('Count')
plt.legend(title='Sex of Casualty', labels=['Male', 'Female'])
plt.show()

2. Age Band vs. Casualty Severity

In [ ]:
sns.countplot(x='age_band_of_casualty', hue='casualty_severity', data=df, order=sorted(df['age_band_of_casualty'].unique()), palette=current_palette)
plt.title('Casualty Severity by Age Band')
plt.xticks(rotation=45)
plt.show()

Plot the relationship between casualty severity and casualty class

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='casualty_severity', hue='casualty_class', data=df, palette=current_palette)
plt.title('Casualty Severity by Casualty Class')
plt.xlabel('Casualty Severity')
plt.ylabel('Count')
plt.legend(title='Casualty Class', labels=['Driver', 'Passenger', 'Pedestrian'])
plt.show()

Plot the relationship between casualty severity and casualty's home area type

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='casualty_severity', hue='casualty_home_area_type', data=df, palette=current_palette)
plt.title('Casualty Severity by Home Area Type')
plt.xlabel('Casualty Severity')
plt.ylabel('Count')
plt.legend(title='Home Area Type', labels=['Unknown', 'Urban', 'Semi-Urban', 'Rural'])
plt.show()

Plot the distribution of casualty types

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(x='casualty_type', data=df, palette=current_palette)
plt.title('Distribution of Casualty Types')
plt.xlabel('Casualty Type')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

Plot the distribution of pedestrian location values

In [ ]:
pedestrian_movement_labels = {
    0: 'Not a Pedestrian',
    1: "Crossing from driver's nearside",
    2: 'Crossing from nearside - masked by parked or stationary vehicle',
    3: "Crossing from driver's offside",
    4: 'Crossing from offside - masked by parked or stationary vehicle',
    5: 'In carriageway, stationary - not crossing (standing or playing)',
    6: 'In carriageway, stationary - not crossing (standing or playing) - masked by parked or stationary vehicle',
    7: 'Walking along in carriageway, facing traffic',
    8: 'Walking along in carriageway, back to traffic',
    9: 'Unknown or other'
}

pedestrian_location_labels = {
    0: 'Not a Pedestrian',
    1: 'Crossing on pedestrian crossing facility',
    2: 'Crossing in zig-zag approach lines',
    3: 'Crossing in zig-zag exit lines',
    4: 'Crossing elsewhere within 50m. of pedestrian crossing',
    5: 'In carriageway, crossing elsewhere',
    6: 'On footway or verge',
    7: 'On refuge, central island or central reservation',
    8: 'In centre of carriageway - not on refuge, island or central reservation',
    9: 'In carriageway, not crossing',
    10: 'Unknown or other'
}

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.countplot(x='pedestrian_location', data=df, ax=axes[0], palette=current_palette)
axes[0].set_title('Distribution of Pedestrian Location Values')
axes[0].set_xlabel('Pedestrian Location')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(pedestrian_location_labels.values(), rotation=90)  # Add legend with rotated labels

sns.countplot(x='pedestrian_movement', data=df, ax=axes[1], palette=current_palette)
axes[1].set_title('Distribution of Pedestrian Movement Values')
axes[1].set_xlabel('Pedestrian Movement')
axes[1].set_ylabel('Count')
axes[1].set_xticklabels(pedestrian_movement_labels.values(), rotation=90)  # Add legend with rotated labels

plt.tight_layout()

plt.show()

Plot the relationship between pedestrian location, pedestrian movement and casualty severity

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# Plot the boxplot of casualty severity by pedestrian location
sns.boxplot(x='pedestrian_location', y='casualty_severity', data=df, ax=axes[0], palette=current_palette)
axes[0].set_title('Casualty Severity by Pedestrian Location')
axes[0].set_xlabel('Pedestrian Location')
axes[0].set_ylabel('Casualty Severity')

# Plot the boxplot of casualty severity by pedestrian movement
sns.boxplot(x='pedestrian_movement', y='casualty_severity', data=df, ax=axes[1], palette=current_palette)
axes[1].set_title('Casualty Severity by Pedestrian Movement')
axes[1].set_xlabel('Pedestrian Movement')
axes[1].set_ylabel('Casualty Severity')

# Add legend
legend_labels = ['Not a Pedestrian', 'Crossing on Pedestrian Crossing Facility', 'Crossing in Zig-zag Approach Lines',
                 'Crossing in Zig-zag Exit Lines', 'Crossing Elsewhere within 50m of Pedestrian Crossing',
                 'In Carriageway, Crossing Elsewhere', 'On Footway or Verge', 'On Refuge, Central Island or Central Reservation',
                 'In Centre of Carriageway - Not on Refuge, Island or Central Reservation', 'In Carriageway, Not Crossing',
                 'Unknown or Other']
legend = plt.legend(legend_labels, title='Pedestrian Location', loc='upper left', bbox_to_anchor=(1, 1))
plt.setp(legend.get_title(), fontsize=12)

plt.tight_layout()

plt.show()


Plot the distribution of accidents across age bands

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(x='age_band_of_casualty', data=df, palette=current_palette)
plt.title('Distribution of Accidents by Age Band of Casualty')
plt.xlabel('Age Band of Casualty')
plt.ylabel('Count')

legend_labels = ['0 - 5', '6 - 10', '11 - 15', '16 - 20', '21 - 25', '26 - 35', '36 - 45', '46 - 55', '56 - 65', '66 - 75', 'Over 75']
legend = plt.legend(legend_labels, title='Age Band of Casualty', loc='upper left', bbox_to_anchor=(1, 1))
plt.setp(legend.get_title(), fontsize=12)

plt.show()


Plot the distribution of vehicle reference values

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(x='vehicle_reference', data=df, palette=current_palette)
plt.title('Distribution of Vehicle Reference Values')
plt.xlabel('Vehicle Reference')
plt.ylabel('Count')
plt.show()

Plot the distribution of Casualty_IMD_Decile values

In [ ]:
# Plot the distribution of Casualty IMD Decile values
plt.figure(figsize=(10, 8))
sns.countplot(x='casualty_imd_decile', data=df, palette=current_palette)
plt.title('Distribution of Casualty IMD Decile Values')
plt.xlabel('Casualty IMD Decile')
plt.ylabel('Count')

# Add legend
legend_labels = ['Most deprived 10%', 'More deprived 10-20%', 'More deprived 20-30%', 'More deprived 30-40%', 
                 'More deprived 40-50%', 'Less deprived 40-50%', 'Less deprived 30-40%', 'Less deprived 20-30%', 
                 'Less deprived 10-20%', 'Least deprived 10%']
legend = plt.legend(legend_labels, title='Casualty IMD Decile', loc='upper left', bbox_to_anchor=(1, 1))
plt.setp(legend.get_title(), fontsize=12)

plt.show()


Plot the distribution of Age_of_Casualty values

In [ ]:
plt.figure(figsize=(10, 8))
sns.histplot(df['age_of_casualty'], bins=20, kde=True, palette=current_palette)
plt.title('Distribution of Age of Casualty')
plt.xlabel('Age of Casualty')
plt.ylabel('Count')
plt.show()

 bus or coach passenger categories

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='bus_or_coach_passenger', data=df, palette=current_palette)

passenger_categories = {
    0: 'Not a bus or coach passenger',
    1: 'Boarding',
    2: 'Alighting',
    3: 'Standing passenger',
    4: 'Seated passenger',
    9: 'Unknown (self-reported)'
}

plt.legend(title='Bus or Coach Passenger', labels=[f'{key}: {value}' for key, value in passenger_categories.items()])

plt.title('Distribution of Bus or Coach Passenger Categories')
plt.xlabel('Bus or Coach Passenger Category')
plt.ylabel('Count')
plt.show()


# relationship comparison

Plot the relationship between casualty severity and the IMD decile

In [ ]:
plt.figure(figsize=(10, 8))
sns.boxplot(x='casualty_severity', y='casualty_imd_decile', data=df, palette=current_palette)
plt.title('Casualty Severity by IMD Decile')
plt.xlabel('Casualty Severity')
plt.ylabel('IMD Decile')
plt.show()

Plot the relationship between casualty severity and age band

In [ ]:
plt.figure(figsize=(10, 8))
sns.boxplot(x='casualty_severity', y='age_of_casualty', data=df, palette=current_palette)
plt.title('Casualty Severity by Age of Casualty')
plt.xlabel('Casualty Severity')
plt.ylabel('Age of Casualty')
plt.show()

In [22]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

df.drop(columns=['lsoa_of_casualty', 'accident_index', 'age_of_casualty'], inplace=True)

# Split the data into features (X) and target variable (y)
X = df.drop(columns=['casualty_severity'])
y = df['casualty_severity']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
pipeline = make_pipeline(StandardScaler(), SVC(random_state=42))

# Define hyperparameters for tuning
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf', 'poly'],
}

# Grid search to find the best combination of hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
best_svc = grid_search.best_estimator_
best_svc.fit(X_train, y_train)

# Predictions on the test set
y_pred = best_svc.predict(X_test)

# Print classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Fitting 5 folds for each of 12 candidates, totalling 60 fits



KeyboardInterrupt



In [24]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RFE with RandomForestClassifier
rfe_selector = RFE(estimator=rf_classifier, n_features_to_select=5, step=1)

# Fit RFE to the training data
rfe_selector = rfe_selector.fit(X_train, y_train)

# Get selected feature names
selected_features = X_train.columns[rfe_selector.support_]

# Print selected feature names
print("Selected Features:")
print(selected_features)


Selected Features:
Index(['age_band_of_casualty', 'pedestrian_location', 'pedestrian_movement',
       'casualty_type', 'casualty_imd_decile'],
      dtype='object')


In [ ]:
df.to_csv('cleaned_data.csv')